<a href="https://colab.research.google.com/github/verma-priyanka/APIs/blob/master/RSProject_GEE_PythonAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install library
!pip install earthengine-api --upgrade

In [2]:
# Import the Earth Engine library.
import ee

# Trigger the authentication flow.
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/tgH_1E7Ocb1_hGy1D9oV6gG1HRVirsBNYCpzdDin75boJlxCwJS9P7Y

Successfully saved authorization token.


ee.Initialize() initializes the API. Required before API is called else a 404 error is thrown

In [0]:
ee.Initialize()

In [26]:
# https://developers.google.com/earth-engine/image_visualization
# stack overflow thread:https://gis.stackexchange.com/questions/342022/trouble-plotting-earth-engine-images-with-folium-using-python
# Import the Folium library 
# Basemap not automatically added when GEE tiles are added
import folium

## This doesn't work anymore
# Define a method for displaying Earth Engine image tiles to folium map
# Establish a connection to Google Maps API for adding tile layers from GEE
# def add_ee_layer(self, eeImageObject, visParams, name):
#   mapID = ee.Image(eeImageObject).getMapId(visParams)
#   folium.raster_layers.TileLayer(
#     tiles = "https://earthengine.googleapis.com/map/"+mapID['mapid']+
#       "/{z}/{x}/{y}?token="+mapID['token'],
#     attr = "Map Data &copy; <a href='https://earthengine.google.com/'>Google Earth Engine</a>",
#     name = name,
#     overlay = True,
#     control = True
#   ).add_to(self)

# This works
def add_ee_layer(self, eeImageObject, visParams, name):
  tiles = ee.data.getTileUrl(eeImageObject.getMapId(visParams), 8014, 4817, 37)
  tiles = tiles[:-12] + "{z}/{x}/{y}"
  folium.raster_layers.TileLayer(
    tiles = tiles,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Location to extract imagery for using geographic coordinates
point = ee.Geometry.Polygon(  [[ -71.972474426004553, 42.602279713467297 ], 
                               [ -71.950438522043598, 42.602367785683853 ], 
                               [ -71.950535365247646, 42.615912367460069 ], 
                               [ -71.972576006685841, 42.615824276665819 ], 
                               [ -71.972474426004553, 42.602279713467297 ]]).buffer(3000)
# Dates
startDate = '2002-04-01'
endDate = '2002-5-31'

# Specify time period of interest
fcc = ee.ImageCollection("LANDSAT/LT05/C01/T1_TOA").filterBounds(point).filterDate(startDate, endDate).min()

# Set visualization parameters
# Check dataset's page for required parameters for visualization
# Define dataset from image collection to visualize in the band list

# ----FALSE COLOR COMPOSITE------
fcc_visParams = {'min': 0,
                 'max': 0.4,
                 'gamma': [0.95, 1.1, 1],
                 'bands': ['B5', 'B4', 'B3']}

# ----TRUE COLOR COMPOSITE------
tcc_visParams = {'min': 0.0,
                 'max': 0.4,
                 'bands':['B4', 'B3', 'B2']}

# Add the tile layer of interest to the folium map
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object
# Specify default location for map
# Specify basemap: Folium defaults to openstreetmap basemap if a tile isn't specified
imageryMap = folium.Map(location=[42.602279713467297, -71.972474426004553], zoom_start=10, height=500, tiles='Stamen Terrain')
imageryMap.add_ee_layer(fcc.clip(point), fcc_visParams, 'False Color Composite')
imageryMap.add_ee_layer(fcc.clip(point), tcc_visParams, 'True Color Composite')

# Add a layer control panel to the map
# Allows you to turn layers on and off
imageryMap.add_child(folium.LayerControl())

# Display the map
display(imageryMap)

In [17]:
task = ee.batch.Export.image.toDrive(image=fcc,
                                     description='myDescription',
                                     folder='eePythonAPI',
                                     fileNamePrefix='myFilePrefix')
task.start()

task.status()


{'creation_timestamp_ms': 1574388667066,
 'description': 'myDescription',
 'id': 'EX6MZJW6Q4IFCNPXOIPO6ESZ',
 'name': 'projects/earthengine-legacy/operations/EX6MZJW6Q4IFCNPXOIPO6ESZ',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1574388667066}

In [0]:
import ee

ee.Initialize()

# Get a download URL for an image.
image1 = ee.Image('srtm90_v4')
path = image1.getDownloadUrl({
    'scale': 30,
    'crs': 'EPSG:4326',
    'region': '[[-120, 35], [-119, 35], [-119, 34], [-120, 34]]'
})
print path

https://earthengine.googleapis.com/api/download?docid=61b84e597d0e45fe9471472fe95ab510&token=2b9000b5083d716646fa30b3ca266759


In [10]:
#!/usr/bin/env python
"""Where operator example.
Select the forest classes from the MODIS land cover image and intersect them
with elevations above 1000m.
"""

import ee
import ee.mapclient

ee.Initialize()
ee.mapclient.centerMap(-113.41842, 40.055489, 6)

elev = ee.Image('CGIAR/SRTM90_V4')
cover = ee.Image('MODIS/051/MCD12Q1/2001_01_01').select('Land_Cover_Type_1')
blank = ee.Image(0)


# Where (1 <= cover <= 4) and (elev > 1000), set the output to 1.
output = blank.where(
    cover.lte(4).And(cover.gte(1)).And(elev.gt(1000)),
    1)

# Output contains 0s and 1s.  Mask it with itself to get rid of the 0s.
result = output.mask(output)

imageryMap = folium.Map(location=[42.2626,-71.8023], zoom_start=7, height=500, tiles='Stamen Terrain')
imageryMap.add_ee_layer(result, {'palette': '00AA00'}, 'False Color Composite')
display(imageryMap)

In [0]:
# https://developers.google.com/earth-engine/image_visualization
# stack overflow thread:https://gis.stackexchange.com/questions/342022/trouble-plotting-earth-engine-images-with-folium-using-python
# Import the Folium library 
# Basemap not automatically added when GEE tiles are added
import folium

## old code
# Define a method for displaying Earth Engine image tiles to folium map
# Establish a connection to Google Maps API for adding tile layers from GEE
# def add_ee_layer(self, eeImageObject, visParams, name):
#   mapID = ee.Image(eeImageObject).getMapId(visParams)
#   folium.raster_layers.TileLayer(
#     tiles = "https://earthengine.googleapis.com/map/"+mapID['mapid']+
#       "/{z}/{x}/{y}?token="+mapID['token'],
#     attr = "Map Data &copy; <a href='https://earthengine.google.com/'>Google Earth Engine</a>",
#     name = name,
#     overlay = True,
#     control = True
#   ).add_to(self)

def add_ee_layer(self, eeImageObject, visParams, name):
  tiles = ee.data.getTileUrl(eeImageObject.getMapId(visParams), 8014, 4817, 37)
  tiles = tiles[:-12] + "{z}/{x}/{y}"
  folium.raster_layers.TileLayer(
    tiles = tiles,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Location to extract imagery for using geographic coordinates
point = ee.Geometry.Point([-71.8023,42.2626])
# Dates
startDate = '2018-01-01'
endDate = '2019-12-31'

# Specify time period of interest
fcc = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA").filterBounds(point).filterDate(startDate, endDate).min()

# Set visualization parameters
# Check dataset's page for required parameters for visualization
# Define dataset from image collection to visualize in the band list

# ----FALSE COLOR COMPOSITE------
fcc_visParams = {'min': 0,
                 'max': 0.5,
                 'gamma': [0.95, 1.1, 1],
                 'bands': ['B5', 'B4', 'B3']}

# ----TRUE COLOR COMPOSITE------
tcc_visParams = {'min': 0.0,
                 'max': 0.4,
                 'bands':['B4', 'B3', 'B2']}

# Add the tile layer of interest to the folium map
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object
# Specify default location for map
# Specify basemap: Folium defaults to openstreetmap basemap if a tile isn't specified
imageryMap = folium.Map(location=[42.2626,-71.8023], zoom_start=7, height=500, tiles='Stamen Terrain')
imageryMap.add_ee_layer(fcc, fcc_visParams, 'False Color Composite')
imageryMap.add_ee_layer(fcc, tcc_visParams, 'True Color Composite')

# Add a layer control panel to the map
# Allows you to turn layers on and off
imageryMap.add_child(folium.LayerControl())

# Display the map
display(imageryMap)